# Download all the depenndencies beforehand

#link to the model files.

> t5 -  https://drive.google.com/drive/folders/1ikwBy0SKgMuUE8tjfnzxPB1xsanD9TRT?usp=sharing

> Pega - https://drive.google.com/drive/folders/1bsHJnV4JwCdaCm5RMgLADYb3sbkXlHex?usp=drive_link




Youtube Caption Generation.

In [ ]:
import re
import csv
from youtube_transcript_api import YouTubeTranscriptApi as yta
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

api_key = "YPUTUBE_API_KEY"

# Playlist URL
playlist_url = "https://www.youtube.com/list=PLot-Xpze53lfOdF3KwpMSFEyfE77zIwiP"

# Extract playlist ID from the URL
playlist_id = re.findall(r"list=([^&]+)", playlist_url)[0]

# Initialize YouTube Data API client
youtube = build("youtube", "v3", developerKey=api_key)

# Retrieve video IDs from the playlist
video_ids = []
next_page_token = None
while True:
    try:
        playlist_items = youtube.playlistItems().list(
            part="contentDetails",
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        video_ids.extend(item["contentDetails"]["videoId"] for item in playlist_items["items"])

        next_page_token = playlist_items.get("nextPageToken")
        if not next_page_token:
            break

    except HttpError as e:
        print("An error occurred while retrieving playlist items:", e)
        break
with open("transcripts.csv", "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Index", "Content"])  # Write header row

    # Retrieve transcripts for each video
    for index, video_id in enumerate(video_ids, start=1):
        print(f"Processing video {index}/{len(video_ids)}")
        try:
            # Get transcript for the current video
            data = yta.get_transcript(video_id)

            if data is not None:
                transcript = ""

                # Extract the text from each caption item
                for value in data:
                    for key, value in value.items():
                        if key == 'text':
                            transcript += value + " "

                # Write the transcript to the CSV file
                writer.writerow([index, transcript])

            else:
                print(f"Transcripts are disabled for video with ID: {video_id}")

        except Exception as e:
            print(f"An error occurred while retrieving transcript for video with ID: {video_id}")
            print(e)


Senetnece Pre-processing

In [ ]:
def preprocess_sentence(sentence):
    sentence = sentence.lower()
    sentence = re.sub(r'[^a-zA-Z\s]', '', sentence)
    sentence = ' '.join(sentence.split())
    tokens = word_tokenize(sentence)
    preprocessed_sentence = ' '.join(tokens)

    return preprocessed_sentence

Other pre processing were also done like lemmatization, stemming, removing stop words to get the insights of the datset.

# Extractive Summary Generation + Some of the summaries are generated by annotation


In [ ]:
for index, row in dfnew.iterrows():
    content = row['Content']
    f=len(content.split())
    doc = nlp(content)
    tokens=[token.text for token in doc]
    word_freq={}
    stop_words= list(STOP_WORDS)
    for word in doc:
      if word.text.lower() not in stop_words:
        if word.text.lower() not in punctuation:
          if word.text not in word_freq.keys():
            word_freq[word.text]= 1
          else:
            word_freq[word.text]+= 1
    x=(word_freq.values())
    a=list(x)
    a.sort()
    max_freq=a[-1]
    max_freq
    for word in word_freq.keys():
      word_freq[word]=word_freq[word]/max_freq
    sent_score={}
    sent_tokens=[sent for sent in doc.sents]
    for sent in sent_tokens:
      for word in sent:
        if word.text.lower() in word_freq.keys():
          if sent not in sent_score.keys():
            sent_score[sent]=word_freq[word.text.lower()]
          else:
            sent_score[sent]+= word_freq[word.text.lower()]
    summary=nlargest(n=int(len(sent_score) *0.3),iterable=sent_score,key=sent_score.get)
    final_summary=[word.text for word in summary]
    f1=[]
    for sub in final_summary:
      f1.append(re.sub('','',sub))
    f2=" ".join(f1)
    dfnew.loc[index, 'summary'] = f2

Function to find the syllogism using the glove embedding

In [ ]:
def unknown_words(text, vocab):
    tokens = word_tokenize(text)
    unknown = [t for t in tokens if t not in vocab.word2vec]
    return unknown

Function to check the scores for the validation

In [ ]:
from nltk.translate import bleu_score
from nltk.translate.bleu_score import SmoothingFunction

def calculate_rouge(hypothesis, reference):
    hypothesis_tokens = hypothesis.split()
    reference_tokens = reference.split()

    smoothie = SmoothingFunction().method7

    rouge_1_score = bleu_score.sentence_bleu([reference_tokens], hypothesis_tokens, weights=(1, 0, 0, 0), smoothing_function=smoothie)
    rouge_2_score = bleu_score.sentence_bleu([reference_tokens], hypothesis_tokens, weights=(0, 1, 0, 0), smoothing_function=smoothie)
    rouge_l_score = bleu_score.sentence_bleu([reference_tokens], hypothesis_tokens, weights=(0, 0, 1, 0), smoothing_function=smoothie)

    return rouge_1_score, rouge_2_score, rouge_l_score

reference_summary = "if you have never designed a system before this is probably the place to start so imagine you have a computer with you in which you have written an algorithm so some code is running on this computer and this code gives out an output now people look at this code and they decide that this is really useful to them so they're ready to use that code now you cannot go around giving your computer to everybody so what you do is you expose your code using some protocol which is going to be running on the Internet and by exposing your cool using something called"
generated_summary = dataset1['summary'][0]

rouge_1, rouge_2, rouge_l = calculate_rouge(generated_summary, reference_summary)

print("ROUGE-1 score:", rouge_1)
print("ROUGE-2 score:", rouge_2)
print("ROUGE-L score:", rouge_l)


# Model Training and bookmark the results using the markov recorder.

In [ ]:
# You can toggle the path for trainig the dataset.
path = '/content/gdrive/MyDrive/cnn_dailymail/train.csv'

# Remove NUll is our dataset
# path = '/content/gdrive/MyDrive/RemoveNULL.csv'
trainning_df = pd.read_csv(path, engine='python', error_bad_lines=False)

MAX_LEN = 512
SUMMARY_LEN = 150
TRAINNING_SIZE = 1000

trainning_df = trainning_df.iloc[0:TRAINNING_SIZE,:].copy()
trainning_article_ls = list(trainning_df['article'])
trainning_highlight_ls = list(trainning_df['highlights'])


In [ ]:
df = pd.DataFrame(columns=['target_text','source_text'])
df['target_text'] = trainning_highlight_ls
df['source_text'] = ['summarize: '+item for item in trainning_article_ls]

In [ ]:
project_id = "3tFHvmKR879osa"
try:
	project = markov.Project.get_by_id(project_id)
except markov.exceptions.ResourceNotFoundException:
  project = markov.Project(project_name)
  project.register()

In [ ]:
recorder = markov.ExperimentRecorder(
    name = "Summarization Tool3_Rogue",
    note = "",
    hyper_parameters={
        "lr": 0.001,
        "momentum": 0.9,
        "batch_size": 4,
        "optimizer": "ADAM"
    },
    project_id=project.project_id
)

In [ ]:
# Code to fine tune the dataset
recorder.register()
with recorder:
  model = SimpleT5()
  model.from_pretrained(model_type="t5", model_name="t5-base")
  MAX_EPOCHS = 2

  torch.cuda.memory_summary(device=None, abbreviated=False)
  torch.utils.checkpoint

  model.train(train_df=df[0:(int)(0.7*TRAINNING_SIZE)],
              eval_df=df[(int)(0.7*TRAINNING_SIZE):TRAINNING_SIZE],
              source_max_token_len=MAX_LEN,
              target_max_token_len=SUMMARY_LEN,
              batch_size=4, max_epochs=MAX_EPOCHS, use_gpu=True)


In [ ]:
with recorder:
    for index in sample(list(np.arange(len(trainning_article_ls))), 5):
        print('Original Text : ')
        print(trainning_article_ls[index])

        Original_article = trainning_article_ls[index]

        print('\n\nSummary Text : ')
        print(trainning_highlight_ls[index])

        Original_Summary = trainning_highlight_ls[index]

        print('\n\nFine tuned Predicted Summary Text : ')
        model.load_model("t5","/content/gdrive/MyDrive/outputs1/simplet5-epoch-1-train-loss-2.844-val-loss-3.2345", use_gpu=True)
        print(model.predict(trainning_article_ls[index]))

        Predicted_Summary = trainning_article_ls[index]

        rouge = Rouge()
        BLEUscore = nltk.translate.bleu_score.sentence_bleu([Predicted_Summary], Original_Summary)

        print("-----------------")

        print(BLEUscore)
        # urid= index
        recorder.add_record({"BLEU Score": BLEUscore})
        recorder.add_record({"Rogue-1 Recall": recall_values_r1[0]})
        recorder.add_record({"Rogue-2 Recall": recall_values_r2[0]})
        recorder.add_record({"Rogue-l Recall": recall_values_rl[0]})
        recorder.add_record({"Rogue-1 Precision": precision_values_r1[0]})
        recorder.add_record({"Rogue-2 Precision": precision_values_r2[0]})
        recorder.add_record({"Rogue-l Precision": precision_values_rl[0]})
        recorder.add_record({"Rogue-1 F1": f_values_r1[0]})
        recorder.add_record({"Rogue-2 F1": f_values_r2[0]})
        recorder.add_record({"Rogue-l F1": f_values_rl[0]})
        print("-----------------")

        print('\n\nNot Fine tuned Predicted Summary Text : ')
        preprocess_text = trainning_article_ls[index].strip().replace("\n","")
        t5_prepared_Text = "summarize: "+preprocess_text
        tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)
        summary_ids = no_tune_model.generate(tokenized_text,
                                            num_beams=4,
                                            no_repeat_ngram_size=2,
                                            min_length=30,
                                            max_length=SUMMARY_LEN,
                                            early_stopping=True)
        output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        print(output)
        print('===========================================================================================================\n\n')

Same way the pegasus is finetuned and trained on the cnn + own dataset, But nit getting the good results.